# Web scraping des annonces d'offres d'emploi sur le site  doopin.net

## Récupération des liens de chaque post

#### Lecture du fichier texte à partir duquel nous allons extraire les lien de chaque offres ainsi que la date de publication

In [330]:
# Lecture du fichier en 1 seul bloc
f2 = open('DooJobs_2023_03_03.txt',"r",encoding="utf8")
content = f2.read()
f2.close()
# affichage de la longueur
print(len(content))
# secion d'une protion pour les expériemtations
texte2=content[-1000:]
texte2

1940649


"oJobs » https://doopin.net/doojSearch\n#job #emploi #icicarecrute\n03/03/2023, 20:25 - +237 6 54 06 66 26: 🔴 🔴 Cliquer sur ce lien bleu 👉 https://doopin.net/IUuou\n«NEW–NOUVEAU» Queen Be Agency & Services – *VENDEURS FREELANCE H/F*\nDélai: 17 Mar.\n\nCette offre ne vous convient pas? Plus d'offres sur DooJobs » https://doopin.net/doojSearch\n#job #emploi #icicarecrute\n03/03/2023, 20:25 - +237 6 54 06 66 26: 🔴 🔴 Cliquer sur ce lien bleu 👉 https://doopin.net/TRswF\n«NEW–NOUVEAU» Queen Be Agency & Services – *TECHNICO COMMERCIAUX*\nDélai: 22 Mar.\n\nCette offre ne vous convient pas? Plus d'offres sur DooJobs » https://doopin.net/doojSearch\n#job #emploi #icicarecrute\n03/03/2023, 20:25 - +237 6 54 06 66 26: 🔴 🔴 Cliquer sur ce lien bleu 👉 https://doopin.net/PWvuY\n«NEW–NOUVEAU» NWJ Consult & Partners – *Assistant Personnel / Personal Assistant*\nContrat: Temporaire/CDD – Délai: 22 Mar.\n\nCette offre ne vous convient pas? Plus d'offres sur DooJobs » https://doopin.net/doojSearch\n#job #e

#### Rcupération du lien de chaque post à l'aide d'une expression regulière construite à cet effet

In [17]:
# importation de la bilbiothèque re pour la manipulation des expressions regulières
import re

In [333]:
# definition et test de l'expression regulière pour l'extraction des posts de nouveaux job
new_job_exp = re.compile(r"([0-9]{2}/[0-9]{2}/[0-9]{2}.+Cliquer sur ce lien bleu 👉 https://doopin.net/[a-zA-Z]{5})")
# print(new_job_exp.findall(texte2))

In [334]:
# utilisation de l'expression regulière pour extraire tous les posts de nouveaux job
dates_and_url = new_job_exp.findall(content)
print(len(dates_and_url))
dates_and_url[:5]

3633


['02/08/2022, 06:15 - +237 6 54 06 66 26: 🔴 🔴 Cliquer sur ce lien bleu 👉 https://doopin.net/fOzsD',
 '02/08/2022, 06:15 - +237 6 54 06 66 26: 🔴 🔴 Cliquer sur ce lien bleu 👉 https://doopin.net/EuUHv',
 '02/08/2022, 06:15 - +237 6 54 06 66 26: 🔴 🔴 Cliquer sur ce lien bleu 👉 https://doopin.net/rhgxW',
 '02/08/2022, 08:37 - +237 6 54 06 66 26: 🔴 🔴 Cliquer sur ce lien bleu 👉 https://doopin.net/RBgwd',
 '02/08/2022, 08:37 - +237 6 54 06 66 26: 🔴 🔴 Cliquer sur ce lien bleu 👉 https://doopin.net/OpTVq']

In [335]:
# definition d'expression regulière pour les dates
dates_exp = re.compile(r"([0-9]{2}/[0-9]{2}/[0-9]{4})")
# definition d'expression regulière pour les url des posts
url_exp = re.compile(r"(https://doopin.net/[a-zA-Z]{5})")

# Récuparation de la date et de l'url de chaque post à partir de la liste récupérée à l'étape précedente
post = {}
posts = []
for elt in dates_and_url:
#     print(elt)
    post['date'] = dates_exp.findall(elt)[0]
    post['url'] = url_exp.findall(elt)[0]
    posts.append(post)
    post={}

print(len(posts))
posts[:5]

3633


[{'date': '02/08/2022', 'url': 'https://doopin.net/fOzsD'},
 {'date': '02/08/2022', 'url': 'https://doopin.net/EuUHv'},
 {'date': '02/08/2022', 'url': 'https://doopin.net/rhgxW'},
 {'date': '02/08/2022', 'url': 'https://doopin.net/RBgwd'},
 {'date': '02/08/2022', 'url': 'https://doopin.net/OpTVq'}]

In [336]:
# Suppression d'éventuel doublon
print('Nombre avant : ',len(posts))
posts = [dict(t) for t in {tuple(d.items()) for d in posts}]
print('Nombre après : ',len(posts))
posts[:5]

Nombre avant :  3633
Nombre après :  3594


[{'date': '03/01/2023', 'url': 'https://doopin.net/sTSQQ'},
 {'date': '14/09/2022', 'url': 'https://doopin.net/gNhrc'},
 {'date': '27/08/2022', 'url': 'https://doopin.net/omltj'},
 {'date': '29/11/2022', 'url': 'https://doopin.net/pNKXQ'},
 {'date': '14/12/2022', 'url': 'https://doopin.net/sHcOk'}]

## Importation des librairies nécessaires au web scapring

In [38]:
# requests pour recupérer des pages depuis internet
import requests
# BeautifulSoup pour parcourir les pages HTML et y extraire les données recherchées 
from bs4 import BeautifulSoup , NavigableString, Tag
# numpy et pandas pour collecter et traiter les données
import numpy as np
import pandas as pd

### Rcupération des informations sur un post d'offre d'emploi

In [40]:
# Rcupération des informations sur un post d'offre d'emploi
date='25/08/2023'
url = 'https://doopin.net/sTSQQ'
response = requests.get(url,timeout=1000) 
soup=BeautifulSoup(response.text,'html.parser')
soup

<!DOCTYPE html>

<html lang="fr">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>SCI SOTRADIC – MAÎTRE NAGEUR  #maitre_nag..</title>
<meta content="SCI SOTRADIC – MAÎTRE NAGEUR  #maitre_nag.." name="title"/>
<meta content="SCI SOTRADIC – MAÎTRE NAGEUR – Douala  #maitre_nageur_sauveteur" name="description"/>
<meta content="" name="keywords"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="false" name="pinterest-rich-pin">
<link href="https://web.doopinet.com?lang=arabic" hreflang="ar" rel="alternate"/><link href="https://web.doopinet.com?lang=bengali" hreflang="bn" rel="alternate"/><link href="https://web.doopinet.com?lang=chinese" hreflang="zh" rel="alternate"/><link href="https://web.doopinet.com?lang=croatian" hreflang="sh" rel="alternate"/><link href="https://web.doopinet.com?lang=danish" hreflang="da" rel="alternate"/><link href="https://web.doopinet.com?lang=dutch" hreflang="nl" rel="alterna

In [41]:
# declaration of a dictionnary to save job offer informations
job_offer = {}
job_offer['lien_emploie']=url
job_offer['date'] = date
job_offer['nom_entreprise']=""
job_offer['secteur_activite']=""
job_offer['metier']=""
job_offer['poste_propose'] =""
job_offer['localisation'] =""
job_offer['delais'] = ""
job_offer['type_de_contrat']=""
job_offer['exigences'] =""

# Get the job title
job = soup.find("div",{"class":"jinfo"})
print('Job title : ', job.h2.text)
job_offer['poste_propose'] =  job.h2.text

# Get the name of the entreprise that offer the job
nom_entreprise = job.find("h4")
job_offer['nom_entreprise']=nom_entreprise.text.replace(" ", "").replace("\n", "").replace("\r", "")

# Get the job details
infos=job.find_all("p",{"class":"wo_vew_apld_ico"})
print(len(infos))

details=""
for elt in infos:
#     print(elt.text.replace(" ", "").replace("\n", ""))
    details = details + elt.text.replace(" ", "").replace("\n", "") + " "
details = details.split()

job_offer['localisation'] = details[0]
job_offer['delais'] = details[1]
job_offer['type_de_contrat']= details[2]
job_offer['secteur_activite']=details[3]
job_offer['metier']= details[4]

#Get the job requirements
exigences = soup.find_all("div",{"class":"edited_text"})
print(len(exigences))

contenus = []
for exigence in exigences:
    contenu=exigence.contents
    contenus = contenus + contenu
# Suppression des doublons
contenus = list(set(contenus))

# Définition d'une expression regulière pour recupérer les informations pertinentes qui commencent par ">>"
criteria_exp=re.compile(r"» [a-zA-Z]+")

vrai_contenus=[]
for i in contenus:
    if isinstance(i, NavigableString):
#         print(i.string.strip())
        vrai_contenus.append(i.string.strip())
    if isinstance(i, Tag):
#         print(i.text.strip())
        vrai_contenus.append(i.text.strip())
#         i.string
# vrai_contenus

# Concaténation de toutes les exigences dans une chaine pour preparaion ultérieure
descriptions_specifique = ""
for i in vrai_contenus:
    descriptions_specifique = descriptions_specifique + i + "|"
descriptions_specifique = descriptions_specifique.replace("||","|")
job_offer['exigences'] = descriptions_specifique

job_offer

Job title :  MAÎTRE NAGEUR – Douala
5
6


{'lien_emploie': 'https://doopin.net/sTSQQ',
 'date': '25/08/2023',
 'nom_entreprise': 'SCISOTRADIC',
 'secteur_activite': 'Sport',
 'metier': 'Maître-nageursauveteur',
 'poste_propose': 'MAÎTRE NAGEUR – Douala',
 'localisation': 'Douala,Littoral,Cameroon',
 'delais': '31Jan.2023',
 'type_de_contrat': 'Permanent/CDI',
 'exigences': "COMPÉTENCES|» Niveau d'expérience: Expérimenté \r\n                                                                                                » Années d'expérience: Entre 5 ans et 8 ans \r\n                                                                                                » Niveau Académique Minimum: Baccalauréat|» Demande d’emploi|PROFIL D'UN MAITRE NAGEUR/PISCINIER|» \r\n                                                                    29 - 40                                                                     ans|CONSTITUTION DOSSIER|Seront transmises à l'adresse suivante\xa0email suivante sotradicrh@gmail.com\xa0ou alors déposées à n

### Rcupération des informations sur tous les posts d'offre d'emploi à partir de leur lien

In [269]:
# Liste qui va contenir toutes offres
jobs_offer = []

In [271]:
for j in range(len(posts)):
    print("Traitement de l'url {}/{}".format(j+1,len(posts)))
    print(ns_posts2[j]['url'])
    response = requests.get(posts[j]['url'], timeout=10000) 
    response.close()
    soup=BeautifulSoup(response.text,'html.parser')
    # declaration of a dictionnary to save job offer informations
    job_offer = {}
    job_offer['lien_emploie'] = posts[j]['url']
    job_offer['date'] = posts[j]['date']
    job_offer['nom_entreprise']=""
    job_offer['secteur_activite']=""
    job_offer['metier']=""
    job_offer['delais']=""
    job_offer['poste_propose'] =""
    job_offer['localisation'] =""
    job_offer['type_de_contrat']=""
    job_offer['exigences'] =""

    try :
        # Get the job title
        job = soup.find("div",{"class":"jinfo"})
        print('Job title : ', job.h2.text)
        job_offer['poste_propose'] =  job.h2.text

        # Get the name of the entreprise that offer the job
        nom_entreprise = job.find("h4")
        job_offer['nom_entreprise']=nom_entreprise.text.replace(" ", "").replace("\n", "").replace("\r", "")

        # Get the job details
        infos=job.find_all("p",{"class":"wo_vew_apld_ico"})
    #     print(len(infos))

        details=""
        for elt in infos:
        #     print(elt.text.replace(" ", "").replace("\n", ""))
            details = details + elt.text.replace(" ", "").replace("\n", "") + " "
        details = details.split()

        job_offer['localisation'] = details[0]
        job_offer['delais'] = details[1]
        job_offer['type_de_contrat']= details[2]
        job_offer['secteur_activite']=details[3]
        job_offer['metier']= details[4]

        #Get the job requirements
        exigences = soup.find_all("div",{"class":"edited_text"})
    #     print(len(exigences))

        contenus = []
        for exigence in exigences:
            contenu=exigence.contents
            contenus = contenus + contenu
        # Suppression des doublons
        contenus = list(set(contenus))

        # Définition d'une expression regulière pour recupérer les informations pertinentes qui commencent par ">>"
        criteria_exp=re.compile(r"» [a-zA-Z]+")

        vrai_contenus=[]
        for i in contenus:
            if isinstance(i, NavigableString):
        #         print(i.string.strip())
                vrai_contenus.append(i.string.strip())
            if isinstance(i, Tag):
        #         print(i.text.strip())
                vrai_contenus.append(i.text.strip())
        #         i.string
        # vrai_contenus

        # Concaténation de toutes les exigences dans une chaine pour preparaion ultérieure
        descriptions_specifique = ""
        for i in vrai_contenus:
            descriptions_specifique = descriptions_specifique + i + "|"
        descriptions_specifique = descriptions_specifique.replace("||","|")
        job_offer['exigences'] = descriptions_specifique
        print("Date : ",job_offer['date'])

        # Ajout de l'offre dans la liste de tous les offres
        jobs_offer.append(job_offer)
    except :
        print("XXXXXXXXXXXXXXXYYYYYYYYYYYYYYY erreur dans cette position YYYYYYYYYYYYYYXXXXXXXXXXXXXXXXXXXXXXX")
        continue
print("Nombre d'offre d'emploi : ",len(jobs_offer))

Traitement de l'url 882/1059
https://doopin.net/SybAh
Job title :  CHAUFFEUR LIVREUR – Douala
Date :  31/12/2022
Traitement de l'url 883/1059
https://doopin.net/QGokm
Job title :  Commercial – Douala
Date :  10/02/2023
Traitement de l'url 884/1059
https://doopin.net/AXPyA
XXXXXXXXXXXXXXXYYYYYYYYYYYYYYY erreur dans cette position YYYYYYYYYYYYYYXXXXXXXXXXXXXXXXXXXXXXX
Traitement de l'url 885/1059
https://doopin.net/cWPTn
Job title :  Aides-Soignants ; 01 ATMS – Douala
Date :  15/02/2023
Traitement de l'url 886/1059
https://doopin.net/BGoKk
Job title :  Accountant – Yaoundé
Date :  17/01/2023
Traitement de l'url 887/1059
https://doopin.net/RwxXI
Job title :  Gérant d'une fabrique de parpaings – Yaoundé
Date :  24/02/2023
Traitement de l'url 888/1059
https://doopin.net/lnuzM
Job title :  Finance Officer (Maternity Cover) – Buea
Date :  11/01/2023
Traitement de l'url 889/1059
https://doopin.net/RAjji
Job title :  Recrutement Personnels infirmiers et paramédicaux – Yaoundé
Date :  11/02/2023

XXXXXXXXXXXXXXXYYYYYYYYYYYYYYY erreur dans cette position YYYYYYYYYYYYYYXXXXXXXXXXXXXXXXXXXXXXX
Traitement de l'url 947/1059
https://doopin.net/jSMnZ
Job title :  ON TRADE SALES REPRESENTATIVES H/F – Douala
Date :  13/02/2023
Traitement de l'url 948/1059
https://doopin.net/DXunw
Job title :  Serveurs/serveuses – Yaoundé
Date :  03/03/2023
Traitement de l'url 949/1059
https://doopin.net/SQcHN
Job title :  Graphic Designer H/F – Douala
Date :  08/02/2023
Traitement de l'url 950/1059
https://doopin.net/PFkfo
Job title :  Responsable d’Agence – Cameroun
Date :  01/03/2023
Traitement de l'url 951/1059
https://doopin.net/dUpiu
Job title :  Contrôleur Qualité – Douala
Date :  10/01/2023
Traitement de l'url 952/1059
https://doopin.net/cgFmR
Job title :  Technicien de telecommunication – Douala
Date :  09/01/2023
Traitement de l'url 953/1059
https://doopin.net/onMLq
Job title :  Information Technology Teacher – Limbé
Date :  10/02/2023
Traitement de l'url 954/1059
https://doopin.net/Ycpev
Job t

Job title :  Serveuse – Douala - Cameroun
Date :  16/02/2023
Traitement de l'url 1011/1059
https://doopin.net/QOirX
Job title :  VÉTÉRINAIRE – Douala
Date :  09/02/2023
Traitement de l'url 1012/1059
https://doopin.net/StxJU
Job title :  CHARGÉ DES RELATIONS INTERNATIONAL – Douala
Date :  18/02/2023
Traitement de l'url 1013/1059
https://doopin.net/wFWqv
Job title :  Agronome – Douala
Date :  25/02/2023
Traitement de l'url 1014/1059
https://doopin.net/JHMds
Job title :  Commerciale free-lance – Douala
Date :  02/02/2023
Traitement de l'url 1015/1059
https://doopin.net/nswuQ
Job title :  Technicien de froid et climatisation ; Technicien plombier ; Technicien électricien – Kribi
Date :  03/03/2023
Traitement de l'url 1016/1059
https://doopin.net/Rvlkd
Job title :  Apporteur d'affaires – Yaoundé
Date :  08/01/2023
Traitement de l'url 1017/1059
https://doopin.net/fflmP
Job title :  Assistant(e) de gestion & Animation – Bafoussam
Date :  18/01/2023
Traitement de l'url 1018/1059
https://doopin

### Enregistrement des informations collectées dans un fichier csv

In [272]:
import pandas as pd
# import datetime
# now = datetime.datetime.now()
# format the datetime to format dd_mm_YY_H_M_S
# dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
# Transform to dataframe
df = pd.DataFrame(jobs_offer,columns=['lien_emploie','date','nom_entreprise','secteur_activite','metier','poste_propose','localisation','delais','type_de_contrat','exigences' ])

#save to csv file
df.to_csv('jobs_offers_from_doopinnet.csv',index=False,encoding='utf-8')
# Affichage de l'entête du dataframe
df.head()

,lien_emploie,date,nom_entreprise,secteur_activite,metier,poste_propose,localisation,delais,type_de_contrat,exigences
0,https://doopin.net/sTSQQ,03/01/2023,SCISOTRADIC,Sport,Maître-nageursauveteur,MAÎTRE NAGEUR – Douala,"Douala,Littoral,Cameroon",31Jan.2023,Permanent/CDI,COMPÉTENCES|» Niveau d'expérience: Expérimenté...
1,https://doopin.net/zPUHV,17/01/2023,VPSSARL/VPSAGENCYRH,Secrétaire-Assistant(e),Assistant(e)Administratif(ve),Assistant Administratif et Comptable – Yaoundé,"Yaoundé,Centre,Cameroon",Postulerauplustôt,Temporaire/CDD,Maîtriser les procédures administratives.|Suiv...
2,https://doopin.net/yipOd,18/01/2023,CalabashMarketing&Media,Art-Design,Designergraphique,Directeur Artistique – Douala,"Douala,Littoral,Cameroon",31Jan.2023,Nonspécifié,DIRECTEUR ARTISTIQUE|Doit être excellent en de...
3,https://doopin.net/GBsED,25/01/2023,Koree,Commerce-Vente-Distribution,Commercial,Customer Success Manager H/F – Douala,"Douala,Littoral,Cameroon",Postulerauplustôt,Nonspécifié,Veuillez cliquer sur le bouton Postuler ci-des...
4,https://doopin.net/fLSom,11/01/2023,DOVVDISTRIBUTIONSARL,Comptabilité-Gestion,Contrôleurdegestion,Contrôleurs de gestion – Yaoundé,"Yaoundé,Centre,Cameroon",Postulerauplustôt,Nonspécifié,"Assiduité , dévouement , intégrité ,Disponibil..."


In [273]:
#Affichage des dimensions du dataframe
print(df.shape)
#Affichage de la queue du dataframe
df.tail(3)

(937, 10)


,lien_emploie,date,nom_entreprise,secteur_activite,metier,poste_propose,localisation,delais,type_de_contrat,exigences
934,https://doopin.net/Vbrjn,23/01/2023,GSERVICES,Hôtellerie-Restauration,Serveur,Serveuse Restaurant – Yaoundé,"Bastos,Yaoundé,Centre,Cameroon",Postulerauplustôt,Nonspécifié,|Salaire 50.000 avec possibilité d'augmentati...
935,https://doopin.net/HrnOd,01/02/2023,AFRICANCAPABILITYNETWORK(ACN),Comptabilité-Gestion,Comptable,Comptable – Douala,"Douala,Littoral,Cameroon",Postulerauplustôt,Nonspécifié,» Niveau Académique Minimum: Master|BACC + 4/5...
936,https://doopin.net/NlhXy,25/01/2023,ANGESECURITY,RessourcesHumaines,Responsabledel’administrationdupersonnel,Responsable Administratif et Service généraux ...,"LOGPOM,Douala,Littoral,Cameroon",15Feb.2023,Permanent/CDI,Discipline et rigueur Intégrité et loyautéProa...


## Néttoyage des données et Analyses exploratoire

In [274]:
import numpy as np
pd.options.display.max_columns
pd.options.display.max_rows
pd.set_option('display.max_colwidth', 50)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Setting limit for string
pd.options.display.max_colwidth

In [ ]:
# Lecture des dataset
jobs = pd.read_csv('jobs_offers_from_doopinnet.csv')

In [280]:
# Affichage des dimensions du dataframe
print(jobs.shape)

(2560, 10)
(937, 10)
(3497, 10)


In [281]:
jobs.info() ;

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3497 entries, 0 to 936
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   lien_emploie      3497 non-null   object
 1   date              3497 non-null   object
 2   nom_entreprise    3497 non-null   object
 3   secteur_activite  3497 non-null   object
 4   metier            3497 non-null   object
 5   poste_propose     3497 non-null   object
 6   localisation      3497 non-null   object
 7   delais            3497 non-null   object
 8   type_de_contrat   3497 non-null   object
 9   exigences         3497 non-null   object
dtypes: object(10)
memory usage: 300.5+ KB


In [282]:
# Suppression des ligne dupliquées
jobs.drop_duplicates(inplace=True)
print(jobs.shape)

(3470, 10)


In [283]:
jobs.head(5)

,lien_emploie,date,nom_entreprise,secteur_activite,metier,poste_propose,localisation,delais,type_de_contrat,exigences
0,https://doopin.net/fOzsD,02/08/2022,SociétéGrandMoulinduCameroun(SGMC),Comptabilité-Gestion,Contrôleurinterne,CONTROLEUR INTERNE – Douala,"Douala,Littoral,Cameroon",31Aug.2022,Nonspécifié,|Les candidatures pourront être déposées à l'a...
1,https://doopin.net/EuUHv,02/08/2022,CIMENCAM,Commerce-Vente-Distribution,Responsabledesventes,SALES SERVICE COORDINATOR – Douala,"Douala,Littoral,Cameroon",07Aug.2022,Nonspécifié,|» Niveau d'expérience: Expérimenté \r\n ...
2,https://doopin.net/rhgxW,02/08/2022,CIMENCAM,Comptabilité-Gestion,Comptable,BACK OFFICE COORDINATOR – Douala,"Douala,Littoral,Cameroon",07Aug.2022,Nonspécifié,|Appel à candidatures INTERNE / EXTERNE|• Cont...
3,https://doopin.net/RBgwd,02/08/2022,EaseTravelServices,Informatique-Electronique-Numérique,Développeur,Product Designer – Douala,"Douala,Littoral,Cameroon",20Aug.2022,Nonspécifié,"|Pour postuler, envoyer CV + Lettre de motivat..."
4,https://doopin.net/OpTVq,02/08/2022,UnEmployeur,Industriel,Ingénieurélectricien,INGÉNIEUR ÉLECTRICIEN – Douala,"Bonabéri,Douala,Littoral,Cameroon",Postulerauplustôt,Nonspécifié,"Cv, copie diplôme et lettre de motivation atte..."


### Convertion de la colonne delais en date au format yyyy-mm-dd ( En cas de delais = postulerauplustot, on affecte à delais la date de publication de l'offre)

In [284]:
for i in range(jobs.shape[0]):
    if(jobs.iloc[i,7]=='Postulerauplustôt' or jobs.iloc[i,7]==np.nan):
        jobs.iloc[i,7]=jobs.iloc[i,1]

In [285]:
print(jobs.shape)
jobs.delais.value_counts().sort_values(ascending=False)

(3470, 10)


30Nov.2022    103
30Sep.2022     72
31Aug.2022     54
31Jan.2023     52
15Oct.2022     44
             ... 
17Dec.2022      1
28/08/2022      1
30Oct.2023      1
30Jul.2023      1
14Mar.2023      1
Name: delais, Length: 429, dtype: int64

In [286]:
# Conversion de la colonne delais au format yyyy-mm-dd
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Jan.","/01/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Feb.","/02/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Mar.","/03/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Apr.","/04/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("May.","/05/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Jun.","/06/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Jul.","/07/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Aug.","/08/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Sep.","/09/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Oct.","/10/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Nov.","/11/"))
jobs['delais'] = jobs['delais'].astype('str').map(lambda x : x.replace("Dec.","/12/"))

In [287]:
# Fonction qui convertie une date au format dd/mm/aaaa en date au format yyyy-mm-dd
def convert_delais(x):
    # format
    format = '%d/%m/%Y'
    # convert from string format to datetime format
    date_time = datetime.datetime.strptime(x, format)
    # get the date from the datetime using date() function
    return date_time.date()

jobs.delais = pd.to_datetime(jobs.delais, format='%d/%m/%Y')
# jobs['delais'].apply(convert_delais)

### Conversion de la date de publication (date) au format yyyy-mm-dd

In [288]:
def convert_date(x):
    # format
    format = '%d/%m/%Y'
    # convert from string format to datetime format
    date_time = datetime.datetime.strptime(x, format)
    # get the date from the datetime using date() function
    return date_time.date()
jobs['date'] = jobs['date'].apply(convert_date)

In [289]:
# vérification du format de la date dans les colonne date et delais
jobs.sample(5)

,lien_emploie,date,nom_entreprise,secteur_activite,metier,poste_propose,localisation,delais,type_de_contrat,exigences
2198,https://doopin.net/yrjEX,2022-12-05,3CLimited,Transport-Logistique,Déclarantendouane,DÉCLARANT PRINCIPAL EN DOUANE – Douala,"Douala,Littoral,Cameroon",2022-12-05,Nonspécifié,|COMPÉTENCES FONDAMENTALES|Cliquer sur le bout...
836,https://doopin.net/PFkfo,2023-03-01,SCBCameroun,Banque-Finance,Directeurd’agence,Responsable d’Agence – Cameroun,"Lieu,Nonspécifié,Cameroon",2023-03-07,Permanent/CDI,Fidéliser et développer le portefeuille des pr...
289,https://doopin.net/WgSij,2023-01-03,CTESSARL,Agriculture,Techniciend'agriculture,Technicien Phytosanitaire – Douala,"Akwa,Douala,Littoral,Cameroon",2023-01-31,Nonspécifié,-Superviser\r\net cordonner toutes les prestat...
1213,https://doopin.net/UCxcB,2022-10-09,MYRH,Publicité-Marketing,Directeurcommercialetmarketing,DIRECTEUR COMMERCIAL – Douala,"Douala,Littoral,Cameroon",2022-10-09,Permanent/CDI,envoyez-nous vos documents de candidature à l'...
2354,https://doopin.net/flHXo,2022-12-14,JoiceBusinessSolution,Internet-Web,CommunityManager,02 ASSISTANTS(ES) COMMUNITY MANAGER – Cameroun,"Lieu,Nonspécifié,Cameroon",2022-12-14,Temporaire/CDD,Joice Business Solutionrecherche02 Assistants(...


In [290]:
jobs.columns

Index(['lien_emploie', 'date', 'nom_entreprise', 'secteur_activite', 'metier',
       'poste_propose', 'localisation', 'delais', 'type_de_contrat',
       'exigences'],
      dtype='object')

In [291]:
# Visualisation de la colonne exigences pour avoir comment y extraire le niveau d'expérience et le diplôme minimum requis
exig = jobs.drop(['lien_emploie', 'date', 'nom_entreprise', 'secteur_activite', 'metier','poste_propose', 'localisation', 'delais', 'type_de_contrat'],axis=1)
# exig = jobs['exigences']
print(exig.shape)
exig.sample(3)
exig0=exig['exigences'][250].values[0]
print(exig0)

(3470, 1)
|• Bonne compréhension des technologies HTML5, CSS3, JavaScript,.|• Fournir du contenu texte, image et vidéo attrayant pour les comptes de médias sociaux|» Niveau d'expérience: Expérimenté|• Aisance à travailler avec le CMS Wordpress et le Plugin Ellementor;|Compétences WordPress :|Compétences Community Manager :|• Répondre aux commentaires et aux demandes des clients en temps opportun|• Organiser et participer à des événements pour créer une communauté et renforcer la notoriété de la marque|• Bonne compréhension des technologies frontales;||» Lettre de Motivation|» Curriculum Vitae|• Définir et mettre en œuvre des campagnes de médias sociaux et de communication pour s'aligner sur les stratégies de marketing|• Surveiller et signaler les commentaires et les avis en ligne|• Expérience dans la conception et le développement de sites Web de conception créative;|Veuillez cliquer sur le bouton Postuler ci-dessous pour soumettre votre candidature|Nous recherchons un Community Manage

In [292]:
# definition d'expression regulière pour le niveau d'expérience
niveau_exp2 = re.compile(r"(Niveau d'expérience: .{8,11})")
niveau_exp2.findall(exig0)

["Niveau d'expérience: Expérimenté"]

In [293]:
print(jobs.shape)

(3470, 10)


### Extraction du nombre d'années d'expérience dans la description

In [294]:
# definition d'expression regulière pour le niveau d'expérience
annees_exp = re.compile(r"(Années d'expérience.{8,22})")

# Récuparation du nnombre d'année d'expériences 
def annee_experience( y ):
    tmp = annees_exp.findall(y)
    x =y.lower()
    if(len(tmp)>0):
        return tmp[0]
    elif('senior' in x or '10 années' in x or '10 ans' in x ):
        return 'Sénior ou Expérimenté'
    elif('expérimenté' in x or 'expérience avérée' in x or 'expérience confirmée' in x or 'entre 2 à 5 ans' in x ):
        return 'Confirmé'
    elif('Débutant' in x or 'Junior' in x or 'Stagiaire' in x):
        return 'Débutant ou Junior'
    elif ('jeune diplômé' in x ):
        return 'Jeune diplômé'
    else:
        return 'Non spécifié'
# exig['exigences'].map( lambda x: annees_exp.findall(x))
jobs['annee_experience']=jobs['exigences'].apply(annee_experience)

In [295]:
tmp=jobs.drop(['lien_emploie','date','nom_entreprise','secteur_activite','metier','poste_propose','localisation','type_de_contrat','exigences','delais'],axis=1)
tmp.sample(10)

,annee_experience
442,Années d'expérience: Entre 2 an et 3 ans|
857,Années d'expérience: Entre 5 ans et 8 ans
174,Années d'expérience: Entre 5 ans et 8 ans
31,Années d'expérience: Entre 2 an et 3 ans|
902,Non spécifié
2477,Confirmé
1883,Non spécifié
190,Non spécifié
1256,Années d'expérience: Entre 3 ans et 5 ans
1502,Non spécifié


### Extraction du niveau scolaire/académique minimum requis dans la description

In [296]:
# # definition d'expression regulière pour les diplômes
# diplome_exp = re.compile(r"(Niveau Académique Minimum:.{3,30})")
# # Récuparation du nnombre diplome minimum
# diplome_exp.findall(exig0)

def diplome_minimum(y):
    x =y.lower()
    if('doctorat' in x ):
        return 'Doctorat'
    elif('ingénieur' in x):
        return 'Ingénieur'
    elif('master' in x or 'master 2' in x or 'master ii' in x or 'bac+5' in x or 'bac +5' in x or 'bac + 5' in x or 'bac+ 5' in x ):
        return 'Master'
    elif('master 1' in x or 'master i' in x or 'bac+4' in x or 'bac +4' in x or 'bac + 4' in x or 'bac+ 4' in x ):
        return 'Master 1'
    elif('licence' in x or 'bac+3' in x or 'bac +3' in x or 'bac + 3' in x or 'bac+ 3' in x ):
        return 'Licence'
    elif('bac+2' in x or 'bac +2' in x or 'bac + 2' in x or 'bac+ 2' in x or 'bts' in x or 'hnd' in x ):
        return 'Bac + 2'
    elif('bac+1' in x or 'bac +1' in x or 'bac + 1' in x or 'bac+ 1' in x ):
        return 'Bac + 1'
    elif('bac' in x ):
        return 'Bac'
    elif('probatoire' in x ):
        return 'Probatoire'
    elif('bepc' in x or 'cap' in x):
        return 'BEPC/Cap'
    elif ('diplôme de qualification professionnel' in x or 'dqp' in x):
        return 'DQP'
    elif ('certificat de qualification professionnelle' in x or 'cqp' in x):
        return 'CQP'
    else:
        return 'Non spécifié'
    
jobs['diplome_minimum2']=jobs['exigences'].apply(diplome_minimum)

In [297]:
tmp=jobs.drop(['lien_emploie','date','nom_entreprise','secteur_activite','metier','poste_propose','localisation','type_de_contrat','exigences','delais'],axis=1)
tmp.sample(10)

,annee_experience,diplome_minimum2
2020,Années d'expérience: Entre 2 an et 3 ans,Licence
2276,Années d'expérience: Entre 2 an et 3 ans,Bac
1391,Non spécifié,Non spécifié
458,Non spécifié,Licence
2543,Années d'expérience: Entre 5 ans et 8 ans,Ingénieur
824,Années d'expérience: Entre 2 an et 3 ans|,Non spécifié
602,Non spécifié,Non spécifié
1885,Non spécifié,Ingénieur
1081,Années d'expérience: Entre 2 an et 3 ans|,Master
722,Non spécifié,Non spécifié


### Extraction de la colonne Ville à partir de la colonne localisation

In [298]:
tmp= jobs['localisation'].str.split(',',n=1,expand=True)
# tmp.sample(16)
jobs['Ville'] = tmp[0]
jobs['Ville'].value_counts()

Douala             1205
Yaoundé            1028
Lieu                317
Douala-Cameroun     273
Akwa                 78
                   ... 
Sangmélima            1
Japoma                1
Nkongsamba            1
ImmeubleJAKO          1
Foumban               1
Name: Ville, Length: 73, dtype: int64

In [306]:
### Fonction pour corriger les villes 
def villes(x):
#     x = y.lower()
    if( x in ['Douala-Cameroun', 'Bonamoussadi', 'Akwa', 'Makepe','Deido','Ndokoti','Japoma','Souza','Yassa','LOGPOM','Bonabéri','Ndogbong','ZoneIndustrielleMagziBassa','Ndogpassi','BonamoussadiSable','Bali','Bonanjo','Citédespalmiers']) :
        return 'Douala'
    elif (x in ['Yaoundé','Nkolbisson','Bastos', 'Av.Kennedy','Biyem-Assi', 'ImmeubleJAKO','BoulevarddelaRépubliquefaceancienISES','Emana','ODZATOTALBORNE10','Damas','Nsimeyong', 'Nsimeyong','Tsinga']):
        return 'Yaoundé'
    elif (x == 'Lieu'):
        return 'Non indiqué'
    else:
        return x

jobs['Ville'] = jobs['Ville'].apply(villes)
jobs['Ville'].value_counts().sort_values(ascending=False)[:]

Douala         1814
Yaoundé        1076
Non indiqué     317
Bafoussam        52
Maroua           23
               ... 
Sangmélima        1
Mbanga            1
Kumba             1
Foumban           1
Mbakaou           1
Name: Ville, Length: 45, dtype: int64

In [307]:
jobs.sample(5)

,lien_emploie,date,nom_entreprise,secteur_activite,metier,poste_propose,localisation,delais,type_de_contrat,exigences,annee_experience,diplome_minimum2,Ville,ville2
664,https://doopin.net/oKSHs,2023-01-23,Gehmit,Internet-Web,CommunityManager,Offre de stage : Community manager – Yaoundé,"Tsinga,Yaoundé,Centre,Cameroon",2023-01-29,StageProfessionnel,Rédaction et publication des contenus ciblés s...,Non spécifié,BEPC/Cap,Yaoundé,Yaoundé
728,https://doopin.net/iPEca,2023-02-06,BAOCash&Carry,Comptabilité-Gestion,Contrôleurdegestion,Contrôleur de gestion F/H – Douala,"Douala,Littoral,Cameroon",2023-02-17,Nonspécifié,NOUS RECRUTONS|» Années d'expérience: Entre 3 ...,Années d'expérience: Entre 3 ans et 5 ans,Master,Douala,Douala
46,https://doopin.net/RFlTi,2022-08-03,ANGESECURITY,Commerce-Vente-Distribution,Chargédeclientèle,CHARGÉ(E) CLIENTÈLE H/F – Douala,"LOGPOM,Douala,Littoral,Cameroon",2022-10-15,Permanent/CDI,Souhaitées : organiser un planning de vente et...,Non spécifié,Bac + 2,Douala,Douala
388,https://doopin.net/BYMkW,2023-02-17,UNHCR,Transport-Logistique,Chauffeur,Driver – Cameroun,"Lieu,Nonspécifié,Cameroon",2023-03-01,Nonspécifié,Essential|Field(s) of Education|Functional Ski...,Années d'expérience: Entre 2 an et 3 ans|,Licence,Non indiqué,Non indiqué
243,https://doopin.net/PMzdy,2022-08-16,STDIGITAL,Internet-Web,CommunityManager,COMMUNITY MANAGER – Douala,"Douala,Littoral,Cameroon",2022-08-16,Nonspécifié,En collaboration avec les équipes et la direct...,Confirmé,BEPC/Cap,Douala,Douala


### Extraction de la colonne Region à partir de la colonne localisation

In [303]:
# def region(y):
#     x = y.lower()
#     if 'adamaoua' in x or 'ngaoundéré' in x:
#         return 'Adamaoua'
#     elif 'northwest' in x or 'bamenda' in x:
#         return 'Nord Ouest'
#     elif x.find('Sud-Ouest') != -1 or 'southwest' in x or 'Sud-Ouest' in x:
#         return 'Sud Ouest'
#     elif 'québec' in x or 'quebec' in x:
#         return 'Québec'
#     elif 'est' in x or 'bertoua' in x:
#         return 'Est'
#     elif 'extremeNorth' in x or 'maroua' in x:
#         return 'Extrême Nord'
#     elif 'garoua' in x or 'north(nord)' in x:
#         return 'Nord'
#     elif 'south(sud)' in x or 'ebolowa' in x or 'kribi' in x:
#         return 'Sud'
#     elif 'west(ouest)' in x or 'ouest' in x or 'bafoussam' in x:
#         return 'Ouest(West)'
#     elif 'littoral' in x or 'douala' in x or 'edea' in x:
#         return 'Littoral'
#     elif 'centre' in x or 'yaounde' in 'x' or 'yaoundé' in 'x':
#         return 'Centre'
#     else:
#         return 'Non indiquée'

In [308]:
jobs = jobs.drop(['ville2'], axis=1, errors='ignore')
jobs.columns

Index(['lien_emploie', 'date', 'nom_entreprise', 'secteur_activite', 'metier',
       'poste_propose', 'localisation', 'delais', 'type_de_contrat',
       'exigences', 'annee_experience', 'diplome_minimum2', 'Ville'],
      dtype='object')

### Construction de la colonne Region à partir de la colonne ville et de ma connaissance des regions et villes du Cameroun

In [309]:
# Définitions de la liste des villes par région disponibles dans les données
littoral = ['Douala','EDEA','Dizangué','Mbanga','Nkongsamba']
centre = ['Yaoundé','Nkoteng1Mbandjock','Mbalmayo','Mfou','Okola','Mbankomo','Yoko','Nanga-Eboko','BAFIA',]
ouest = ['Bafoussam','Dschang','Bana','Foumbot','Foumban','Mbouda','Bangangté']
extreme_nord = ['Maroua','Kousseri','Mokolo','Yagoua',]
sud = ['Kribi','Ebolowa','Campo','Sangmélima',]
nord = ['Garoua','Faro','Figuil']
est = ['Bertoua','Yokadouma','Batouri']
nord_ouest = ['Bamenda']
sud_ouest = ['Limbé','Buea','Kumba',]
adamaoua = ['Meiganga','Ngaoundéré','Mbakaou',]
non_indiqué = ['Non indiquée']
quebec_distance = ['Montréal','Travailàdomicile']

# Création d'une colonne vide
jobs['Region'] = np.nan

for i in range(jobs.shape[0]):
    if jobs.iloc[i,12] in adamaoua:
        jobs.iloc[i,13] = 'Adamaoua'
    elif jobs.iloc[i,12] in centre:
        jobs.iloc[i,13] = 'Centre'
    elif jobs.iloc[i,12] in littoral:
        jobs.iloc[i,13] = 'Littoral'
    elif jobs.iloc[i,12] in ouest:
        jobs.iloc[i,13] = 'Ouest'
    elif jobs.iloc[i,12] in quebec_distance:
        jobs.iloc[i,13] = 'Québec'
    elif jobs.iloc[i,12] in est:
        jobs.iloc[i,13] = 'Est'
    elif jobs.iloc[i,12] in extreme_nord:
        jobs.iloc[i,13] = 'Extrême-Nord'
    elif jobs.iloc[i,12] in nord:
        jobs.iloc[i,13] = 'Nord'
    elif jobs.iloc[i,12] in sud:
        jobs.iloc[i,13] = 'Sud'
    elif jobs.iloc[i,12] in nord_ouest:
        jobs.iloc[i,13] = 'Nord-Ouest'
    elif jobs.iloc[i,12] in sud_ouest:
        jobs.iloc[i,13] = 'Sud-Ouest'
    else:
        jobs.iloc[i,13] = 'Non indiquée'

In [310]:
print(jobs.shape)
jobs.sample(5)

(3470, 14)


,lien_emploie,date,nom_entreprise,secteur_activite,metier,poste_propose,localisation,delais,type_de_contrat,exigences,annee_experience,diplome_minimum2,Ville,Region
238,https://doopin.net/vaqVn,2023-03-02,ADVANSCAMEROUN,Commerce-Vente-Distribution,Chargédeclientèle,Chargé Clientèle – Cameroun,"Lieu,Nonspécifié,Cameroon",2023-03-02,Nonspécifié,|Vous souhaitez rejoindre la Microfinance lead...,Confirmé,Non spécifié,Non indiqué,Non indiquée
869,https://doopin.net/qUCXs,2022-09-19,KSCSarl,Hôtellerie-Restauration,Réceptionniste,RÉCEPTIONNISTE-COURSIER(ÈRE) BILINGUE ET ASSIS...,"Douala,Littoral,Cameroon",2022-09-30,Nonspécifié,Délai le 30 Septembre 2022|Cliquer sur le bout...,Non spécifié,Non spécifié,Douala,Littoral
580,https://doopin.net/JknhB,2022-09-02,CIMAFCAMEROUN,Commerce-Vente-Distribution,Responsablecommercial,"Responsables Commerciaux – Douala, Bafoussam","Douala,Bafoussam",2022-09-30,Permanent/CDI,» Niveau d'expérience: Senior|Les postes sont ...,Années d'expérience: Entre 5 ans et 8 ans,Ingénieur,Douala,Littoral
2485,https://doopin.net/MzCyU,2022-12-26,GEM,Commerce-Vente-Distribution,Assistantcommercial,ASSISTANT COMMERCIAL – Yaoundé,"Yaoundé,Centre,Cameroon",2023-01-02,Temporaire/CDD,|Assistant(e) commercial(e) En étroite collab...,Années d'expérience: Entre 2 an et 3 ans,Bac + 2,Yaoundé,Centre
2172,https://doopin.net/KGyVq,2022-12-02,WOURITV,Internet-Web,CommunityManager,Community Managers ; Commerciaux – Douala,"Douala,Littoral,Cameroon",2022-12-02,StageProfessionnel,Si vous êtes à la recherche d'un stage dans le...,Non spécifié,Non spécifié,Douala,Littoral


### Extraction de la colonne Pays à partir de la colonne localisation

In [311]:
def pays(y):
    x = y.lower()
    if 'canada' in x :
        return 'Canada'
    else:
        return 'Cameroun'

In [312]:
print(jobs.shape)
jobs['Pays'] = jobs['localisation'].apply(pays)
jobs['Pays'].value_counts()

(3470, 14)


Cameroun    3466
Canada         4
Name: Pays, dtype: int64

In [315]:
jobs.sample(3)

,lien_emploie,date,nom_entreprise,secteur_activite,metier,poste_propose,localisation,delais,type_de_contrat,exigences,annee_experience,diplome_minimum2,Ville,Region,Pays
2202,https://doopin.net/yzsKM,2022-12-06,RightPerformance,Commerce-Vente-Distribution,Vendeur/Hôtessedevente,03 HÔTESSES – Cameroun,"Lieu,Nonspécifié,Cameroon",2022-12-07,Temporaire/CDD,Pour le compte d’un client (distributeur de ma...,Confirmé,Bac,Non indiqué,Non indiquée,Cameroun
1929,https://doopin.net/PCxhw,2022-11-19,boulangerieSteed'SBake,Commerce-Vente-Distribution,Commercial,Commercial(e) Bilingue – Cameroun,"Lieu,Nonspécifié,Cameroon",2022-11-27,Nonspécifié,Dossier de candidature|Avoir une expérience en...,Non spécifié,Non spécifié,Non indiqué,Non indiquée,Cameroun
915,https://doopin.net/tEInv,2023-03-03,AURERH,Informatique-Electronique-Numérique,Administrateurdebasededonnées,Administrateur de base de données F/H – Douala...,"Douala-Cameroun,Littoral,Cameroon",2023-03-20,Permanent/CDI,·|· \r\nExpérience\r\nprofessionnelle: ...,Années d'expérience: Entre 2 an et 3 ans,Master,Douala,Littoral,Cameroun


### Extraction du niveau d'expérience à partir du nombre d'année d'expérience

In [316]:
# Formatage de la colonne annee_experience
jobs['annee_experience'] = jobs['annee_experience'].str.replace("Années d'expérience: ",'')
jobs['annee_experience'] = jobs['annee_experience'].str.replace("|",'')
jobs['annee_experience'] = jobs['annee_experience'].str.replace("Entre ",'')
jobs['annee_experience'] = jobs['annee_experience'].str.replace(" ",'')
jobs['annee_experience'] = jobs['annee_experience'].str.replace('\W', '', regex=True)

<ipython-input-316-4ddb322ecafa>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  jobs['annee_experience'] = jobs['annee_experience'].str.replace("|",'')


In [317]:
# Fonction qui renvoie le niveau d'expérience en fonction de la valeur de la colonne annee_experience
def niveau_experience(y):
    # Formatage pour faciliter la recherche
    x = y.replace("Années d'expérience: ",'')
    x = x.replace("|",'')
    x = x.replace("Entre ",'')
    x = x.replace(" ",'')
#     x = x.str.lower()
    # Débutant (moins de 2 ans)
    debutants = ["débutant","Débutant","1anet2ans","Moinsd'unanÊtret","Moinsd'unanNOUS","1anet2ans" , "Moinsdunan" , "Jeunediplômé" , "MoinsdunanNOUS" , "MoinsdunanAusei" , "MoinsdunanNive" , "MoinsdunanSousl" , "MoinsdunanÊtret" , "oinsdunanNiveau" , "MoinsdunanAssure" , "MoinsdunanNB" , "1anCliquezsurle" , "MoinsdunanFormat" , "MoinsdunanToutes" , "AnnéesdexpériencesMinimum6mois" , "MoinsdunanDéve" , "MoinsdunanTousl" ]
    #Junior (Entre 2 et 4 ans)
    juniors = ["junior","Junior","3anset5ans","2anet3ans","Annéesd'expérience:0à5Dossierdeca","3anset5ans", "2anet3ans",  "Annéesdexpérience05ansTranche" , "Annéesdexpérience0à5Acc" , "Annéesdexpérience0à5Dossierdeca" , ]
    #Confirmé (Entre 5 et 9 ans)
    confirmes = ["Confirmé","confirmé","5anset8ans","2anet3ans",]
    #Seniors ou Expérimentés ( 10 ans ou plus)
    seniors = ["SéniorouExpérimenté","Sénior","expérimenté","sénior","Expérimenté","8anset11an","Plusde15ans" , "Plusde15ansCur" , "Plusde15ansLieu" , "Plusde15ansNiv"]
    if (x in debutants):
        return 'Débutant'
    elif( x in juniors):
        return 'Junior'
    elif (x in confirmes):
        return 'Confirmé'
    elif(x in seniors):
        return 'Sénior ou Expérimenté'
    else:
        return 'Non indiqué'

In [318]:
jobs['annee_experience2'] = jobs['annee_experience'].apply(niveau_experience)

In [319]:
jobs['annee_experience2'].value_counts()

Non indiqué              1539
Junior                    951
Confirmé                  577
Débutant                  320
Sénior ou Expérimenté      83
Name: annee_experience2, dtype: int64

In [320]:
jobs.diplome_minimum2.value_counts()

Non spécifié    909
Licence         648
Bac + 2         515
Master          481
BEPC/Cap        394
Bac             277
Ingénieur       165
Doctorat         25
Probatoire       22
Master 1         15
Bac + 1          13
DQP               6
Name: diplome_minimum2, dtype: int64

In [321]:
jobs.metier.value_counts()

Commercial                   310
Comptable                    196
Autres                       125
Développeur                  119
Assistant(e)dedirection      101
                            ... 
Secrétairededocumentation      1
Directeurdeclientèle           1
Développeurbasededonnées       1
Gestionnairedepaie             1
Assistantematernelle           1
Name: metier, Length: 332, dtype: int64

In [323]:
jobs.sample(3)

,lien_emploie,date,nom_entreprise,secteur_activite,metier,poste_propose,localisation,delais,type_de_contrat,exigences,annee_experience,diplome_minimum2,Ville,Region,Pays,annee_experience2
365,https://doopin.net/PfoEs,2022-08-24,YMCACAMEROUN,Hôtellerie-Restauration,Gérant,"GÉRANT, CUISINIER, SERVEUR – Yaoundé","Yaoundé,Centre,Cameroon",2022-09-20,Nonspécifié,COMPOSITION DU DOSSIER :Une lettre de motivati...,Nonspécifié,Bac,Yaoundé,Centre,Cameroun,Non indiqué
382,https://doopin.net/PHWTc,2022-08-25,AFRICAGASTRONOMIQUEGROUP,Hôtellerie-Restauration,Maîtred’hôtel,RECRUTEMENT MASSIF: RESTAURATION – Yaoundé,"Yaoundé,Centre,Cameroon",2022-09-10,Nonspécifié,Open interview:|Vu le nombre de personnes à re...,Nonspécifié,Non spécifié,Yaoundé,Centre,Cameroun,Non indiqué
292,https://doopin.net/VpMQK,2022-08-18,ETSEXTRANATURE,Agriculture,Jardinier-paysagiste,JARDINIER QUALIFIÉ – Douala,"Douala,Littoral,Cameroon",2022-08-25,Permanent/CDI,"Tonte, fauchage, élagage et taille\r\n\r\nEntr...",Confirmé,BEPC/Cap,Douala,Littoral,Cameroun,Confirmé


In [325]:
# Suppression des solonnes non utiles pour la suite de nos analyses ; elles nous ont juste servi pour la preparation des données
jobs.drop(['lien_emploie','exigences','niveau_experience'],axis=1,inplace=True, errors='ignore')

In [326]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3470 entries, 0 to 936
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               3470 non-null   object        
 1   nom_entreprise     3470 non-null   object        
 2   secteur_activite   3470 non-null   object        
 3   metier             3470 non-null   object        
 4   poste_propose      3470 non-null   object        
 5   localisation       3470 non-null   object        
 6   delais             3470 non-null   datetime64[ns]
 7   type_de_contrat    3470 non-null   object        
 8   annee_experience   3470 non-null   object        
 9   diplome_minimum2   3470 non-null   object        
 10  Ville              3470 non-null   object        
 11  Region             3470 non-null   object        
 12  Pays               3470 non-null   object        
 13  annee_experience2  3470 non-null   object        
dtypes: dateti

In [163]:
# plt.figure(figsize=(12,18))
# jobs.annee_experience.value_counts().sort_values(ascending=True).plot(kind='barh');
# plt.ylabel("secteur d'activité");
# plt.xlabel("Nombre d'offres") ;
# plt.title("Répartition des offres d'emploi par annee_experience au CMR pour {} offres d'emploi recensées".format(jobs.shape[0])) ;
# # plt.savefig('nombre_offre_par_experience_h.png');
# # plt.savefig('nombre_offre_par_secteur_activité_h.jpg');

In [162]:
# plt.figure(figsize=(12,18))
# jobs.diplome_minimum2.value_counts().sort_values(ascending=True).plot(kind='barh');
# plt.ylabel("secteur d'activité");
# plt.xlabel("Nombre d'offres") ;
# plt.title("Répartition des offres d'emploi par diplome_minimum au CMR pour {} offres d'emploi recensées".format(jobs.shape[0])) ;
# plt.savefig('nombre_offre_par_diplome_minimum_h.png');
# # plt.savefig('nombre_offre_par_secteur_activité_h.jpg');

### Enregistrement du jeu de données recupéré et nettoyé 

In [328]:
#save the final dataset to csv file 
# now = datetime.datetime.now()
# format the datetime to format dd_mm_YY_H_M_S
# dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
jobs.to_csv('jobs_offers_clean.csv',index=False,encoding='utf-8')